# Projeto Airbnb Rio - Ferramenta de Previsão de Preço de Imóvel para pessoas comuns 

## Introdução

### Contexto

No Airbnb, qualquer pessoa que tenha um quarto ou um imóvel de qualquer tipo (apartamento, casa, chalé, pousada, etc.) pode ofertar o seu imóvel para ser alugado por diária.

Você cria o seu perfil de host (pessoa que disponibiliza um imóvel para aluguel por diária) e cria o anúncio do seu imóvel.

Nesse anúncio, o host deve descrever as características do imóvel da forma mais completa possível, de forma a ajudar os locadores/viajantes a escolherem o melhor imóvel para eles (e de forma a tornar o seu anúncio mais atrativo)

Existem dezenas de personalizações possíveis no seu anúncio, desde quantidade mínima de diária, preço, quantidade de quartos, até regras de cancelamento, taxa extra para hóspedes extras, exigência de verificação de identidade do locador, etc.

### Nosso objetivo

Construir um modelo de previsão de preço que permita uma pessoa comum que possui um imóvel possa saber quanto deve cobrar pela diária do seu imóvel.

Ou ainda, para o locador comum, dado o imóvel que ele está buscando, ajudar a saber se aquele imóvel está com preço atrativo (abaixo da média para imóveis com as mesmas características) ou não.

### O que temos disponível, inspirações e créditos

As bases de dados foram retiradas do site kaggle: https://www.kaggle.com/allanbruno/airbnb-rio-de-janeiro

Elas estão disponíveis para download abaixo da aula (se você puxar os dados direto do Kaggle pode ser que encontre resultados diferentes dos meus, afinal as bases de dados podem ter sido atualizadas).

Caso queira uma outra solução, podemos olhar como referência a solução do usuário Allan Bruno do kaggle no Notebook: https://www.kaggle.com/allanbruno/helping-regular-people-price-listings-on-airbnb

Você vai perceber semelhanças entre a solução que vamos desenvolver aqui e a dele, mas também algumas diferenças significativas no processo de construção do projeto.

- As bases de dados são os preços dos imóveis obtidos e suas respectivas características em cada mês.
- Os preços são dados em reais (R$)
- Temos bases de abril de 2018 a maio de 2020, com exceção de junho de 2018 que não possui base de dados

### Expectativas Iniciais

- Acredito que a sazonalidade pode ser um fator importante, visto que meses como dezembro costumam ser bem caros no RJ
- A localização do imóvel deve fazer muita diferença no preço, já que no Rio de Janeiro a localização pode mudar completamente as características do lugar (segurança, beleza natural, pontos turísticos)
- Adicionais/Comodidades podem ter um impacto significativo, visto que temos muitos prédios e casas antigos no Rio de Janeiro

Vamos descobrir o quanto esses fatores impactam e se temos outros fatores não tão intuitivos que são extremamente importantes.

## Códigos

### Importar Bibliotecas e Bases de Dados

In [1]:
import pandas as pd
import pathlib
import warnings # Biblioteca usada para ocultar os avisos do Executor do Terminal

warnings.filterwarnings('ignore')
warnings.warn('DelftStack')
warnings.warn('Do not show this message')

### Consolidar Base de Dados

In [2]:
# Dicionário criado para podermos referenciar o mês ao nome dos arquivos
meses = {'jan':1, 'fev':2, 'mar':3, 'abr':4, 'mai':5, 'jun':6, 'jul':7 , 'ago':8, 'set': 9, 'out':10, 'nov':11, 'dez':12}

caminho_bases = pathlib.Path('dataset') #nome da pasta

base_airnb = pd.DataFrame() #daframe vazio que será alimentado pelo append mais abaixo no for

for arquivo in caminho_bases.iterdir(): # fazendo a iteração em todos os arquivos dentro da pasta
    nome_mes = arquivo.name[:3] # pegando os três primeiros caracteres do nome do arquivo
    mes = meses[nome_mes] # trazendo o número do mês conforme a chave do dicionário

    ano = arquivo.name[-8:] # trazendo os oito últimos caracteres do nome do arquivo
    ano = int(ano.replace('.csv', '')) # substituindo o nome da extensão por vazio para restar apenas o ano

    df = pd.read_csv(caminho_bases / arquivo.name)
    df['ano'] = ano
    df['mes'] = mes
    base_airnb = base_airnb.append(df)

In [3]:
base_airnb.head(2)

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,number_of_reviews_ltm,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms
0,14063,https://www.airbnb.com/rooms/14063,20180713234102,2018-07-14,Living in a Postcard,"Besides the most iconic's view, our apartment ...",NaN,"Besides the most iconic's view, our apartment ...",none,Best and favorite neighborhood of Rio. Perfect...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,17878,https://www.airbnb.com/rooms/17878,20180713234102,2018-07-14,Very Nice 2Br - Copacabana - WiFi,Please note that special rates apply for New Y...,- large balcony which looks out on pedestrian ...,Please note that special rates apply for New Y...,none,This is the best spot in Rio. Everything happe...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Se tivermos muitas colunas, já vamos identificar quais colunas podemos excluir

In [4]:
print(base_airnb.columns)
# Gerando um arquivo csv com as primeiras 1000 linhas para entendermos quais colunas excluir
base_airnb.head(1000).to_csv('exportacoes_lab/primeiros_mil_registros.csv', sep=';', index=False)

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'summary',
       'space', 'description', 'experiences_offered', 'neighborhood_overview',
       ...
       'minimum_minimum_nights', 'maximum_minimum_nights',
       'minimum_maximum_nights', 'maximum_maximum_nights',
       'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm',
       'number_of_reviews_ltm', 'calculated_host_listings_count_entire_homes',
       'calculated_host_listings_count_private_rooms',
       'calculated_host_listings_count_shared_rooms'],
      dtype='object', length=108)


- Tendo em vista que existem muitas colunas no dataset, este modelo pode acabar muito lento.
- Além disso, uma análise mais rápida permite ver que várias colunas não são necessárias para o nosso modelo de previsão, por isso, vamos excluir algumas colunas da nossa base.
- Tipos de colunas que vamos excluir:
    1. IDs, links e informações não relevantes para o modelo preditivo;
    2. Colunas repetidas ou extremamente parecidas com outra (que dão a mesma informação para o modelo. Ex "data" e "ano/mes");
    3. Colunas preenchidas com texto livre -> Não rodaremos nenhuma análise de palavras ou mineração de texto;
    4. Colunas em que todos ou quase todos os valores são iguais
- Para isso, vamos criar um arquivo em excel com os primeiros mil registros para analisarmos detalhamente cada coluna.

In [5]:
# Verificando quantos itens temos por categoria nesta coluna do dataframe
print(base_airnb[['experiences_offered']].value_counts())

experiences_offered
none                   902210
dtype: int64


In [6]:
# comparando se os valores das colunas são iguais ou na maioria são iguais
print((base_airnb['host_listings_count']==base_airnb['host_total_listings_count']).value_counts())

True     901750
False       460
dtype: int64


In [8]:
columns = ['host_response_time','host_response_rate','host_is_superhost','host_listings_count','latitude','longitude','property_type','room_type','accommodates','bathrooms','bedrooms','beds','bed_type','amenities','price','security_deposit','cleaning_fee','guests_included','extra_people','minimum_nights','number_of_reviews','review_scores_rating','review_scores_accuracy','review_scores_cleanliness','review_scores_checkin','review_scores_communication','review_scores_location','review_scores_value','instant_bookable','is_business_travel_ready','cancellation_policy','ano','mes']

In [ ]:
base_airnb_uteis = base_airnb.loc[:,columns]

### Tratar Valores Faltando

### Verificar Tipos de Dados em cada coluna

### Análise Exploratória e Tratar Outliers

### Encoding

### Modelo de Previsão

### Análise do Melhor Modelo

### Ajustes e Melhorias no Melhor Modelo